# CDR Generator

In [1]:
import pandas as pd
import random
import math
from datetime import timedelta, datetime

## Loading contact list
Here a text file containing 100 contacts is loaded. The numbers are accorsing to Indian Phone Nuumbers Standards

In [2]:
contact_df = pd.read_fwf('file.txt', sep="\n", header=0, names = ['Contact Numbers'])
contact_df.drop([0], inplace=True)
#contact_df.rename(columns = {0:'Contacts'}, inplace = True) 
contact_df.head()

,Contact Numbers
1,+91-985-556-8744
2,+91-935-551-4172
3,+91-855-524-2371
4,+91-855-579-5317
5,+91-915-557-2013


# Functions to Generate a valid IMEI for Target Number

In [3]:
def luhn_residue(digits):
    return sum(sum(divmod(int(d)*(1 + i%2), 10))
                 for i, d in enumerate(digits[::-1])) % 10

In [4]:
def getImei(N):
    part = ''.join(str(random.randrange(0,9)) for _ in range(N-1))
    res = luhn_residue('{}{}'.format(part, 0))
    return '{}{}'.format(part, -res%10)

In [5]:
def get_Imei_list(number_of_imeis, number_of_logs):
    temp_list = []
    
    for i in range(number_of_imeis):
        temp_list.append(str(getImei(15)))
        
    IMEI_list = []
    for _ in range(number_of_logs):
        IMEI_list.append(random.choice(temp_list))
    
    return IMEI_list

# Function to generate Callers or Called List

In [6]:
def called_numbers(number_of_logs):
    call_list = []
    for i in range(number_of_logs):
        call_list.append(random.choice(list(contact_df['Contact Numbers'])))
    return call_list

# Generating Call Duration Logs

In [7]:
def random_start_call(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [8]:
def random_end_call(start):
    return start + timedelta(minutes=random.randrange(60), seconds=random.randrange(60))

In [9]:
#start_stamp = datetime.strptime('1/1/2020 1:30 PM', '%m/%d/%Y %I:%M %p')
#end_stamp = datetime.strptime('3/10/2020 4:50 AM', '%m/%d/%Y %I:%M %p')

def call_durations(cdr_start_stamp, cdr_end_stamp, number):
    call_durations={
        'call_start_stamp':[],
        'call_end_stamp':[],
    }
    
    start_stamp = datetime.strptime(cdr_start_stamp, '%m/%d/%Y %I:%M %p')
    end_stamp = datetime.strptime(cdr_end_stamp, '%m/%d/%Y %I:%M %p')
    
    for _ in range(number):
        call_durations['call_start_stamp'].append((random_start_call(start_stamp, end_stamp)))
        
    for i in call_durations['call_start_stamp']:
        call_durations['call_end_stamp'].append(random_end_call(i))
    
    return call_durations

# Generating Random Locations for Analysis

In [10]:
def random_locations(flag, n):
    calling_location_list = []
    
    if flag == 'Y'  or flag == 'y':
        
        locations = [
            (28.881342, 78.581894),
            (28.892516, 78.725159),
            (28.847369, 78.758552),
            (28.829659, 78.715692),
            (28.792365, 78.756622),
            (28.696474, 78.660419)    
        ]

        for i in range(n):
            calling_location_list.append(random.choice(locations))
            
    return calling_location_list

# Function to Generate CDR in CSV and JSON format

In [11]:
def generate_cdr(IMEI_list, target_number_list, call_list, call_durations, calling_location_list, n):
    
    cdr_df = pd.DataFrame({
        'IMEI_Number': IMEI_list,
        'Phone_Number': target_number_list,
        'Called_Numbers': call_list,
        'Type': [random.choice(['INCOMING','OUTGOING']) for i in range(n)],
        'Call_Start_Time': call_durations['call_start_stamp'],
        'Call_End_Time': call_durations['call_end_stamp'],
        'Location': calling_location_list
    })
    
#    cdr_df.to_csv('CDR for {}.csv'.format(target_number_list[0]))
#    cdr_df.to_json('CDR for {}.json'.format(target_number_list[0]))
    cdr_df.to_excel('CDR for {}.xlsx'.format(target_number_list[0]))
    print(cdr_df)

# Main Function

In [12]:
def main():
    target_number =  '+91-'+input("Enter the phone number(without country code): ")
    number_of_logs = int(input('Enter the number of call logs to generate: '))
    target_number_list = [str(target_number) for i in range(number_of_logs)]
    called_list = called_numbers(number_of_logs)
    number_of_imeis = int(input('Enter Number of random IMEI for {}: '.format(target_number)))
    imei_list = get_Imei_list(number_of_imeis, number_of_logs)
    
    cdr_start_stamp = input('''
    Enter the start date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    Start Date: 
    ''')
    
    cdr_end_stamp = input('''
    Enter the end date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    End Date: 
    ''')
    
    call_duration_dict=call_durations(cdr_start_stamp, cdr_end_stamp, number_of_logs)
    
    flag = input('Do you want pre-set random locations(Y/N): ')
    location_list = random_locations(flag, number_of_logs)
    generate_cdr(imei_list, target_number_list, called_list, call_duration_dict, location_list, number_of_logs)

In [13]:
# Call this cell to start generating your CDR
main()

Enter the phone number(without country code): 9837880059
Enter the number of call logs to generate: 25000
Enter Number of random IMEI for +91-9837880059: 5

    Enter the start date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    Start Date: 
    11/1/2019 12:00 AM

    Enter the end date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    End Date: 
    5/31/2020 12:00 AM
Do you want pre-set random locations(Y/N): Y
           IMEI_Number    Phone_Number    Called_Numbers      Type  \
0      222712125174617  +91-9837880059  +91-905-557-7383  INCOMING   
1      460246720163254  +91-9837880059  +91-925-557-3998  OUTGOING   
2      222712125174617  +91-9837880059  +91-855-579-8994  OUTGOING   
3      276470320767827  +91-9837880059  +91-755-541-2295  INCOMING   
4      276470320767827  +91-9837880059  +91-855-591-9650  OUTGOING   
...                ...             ...               ...       

# Summary
- This CDR generator is a reliable generator that produces CDR on User reuirements.
- CDR includes the following features:-
        - IMEI Number
        - Phone Number (Target Number)
        - Called/Caller Number
        - Call Start Stamp
        - Call End Stamp
        - Location of the Call(Lat & Long Coordinates)
- Locations are by default genrated on hard coded coordinates.

## Drawbacks
[ ] This generator seriously require an algorithm to produuce random location coordinates according to user needs keeping intact the analysis approach.

[ ] This Generator can not produce multiple CDRs (linked in some way with same contacts).

# OPEN FOR SUGGESTION AND IMPROVEMENTS.

![alt text](https://mtv.mtvnimages.com/uri/mgid:file:http:shared:mtv.com/news/wp-content/uploads/2015/12/suggestions-1449024387.gif?quality=.8&height=294&width=540)